<a href="https://colab.research.google.com/github/essat20/NLP_CW_210021102/blob/main/NLP_Prompt_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PROMPT ENGINEERING

In [4]:
# downloading datasets
!pip install datasets # install the datasets from huggingface
!pip install -q -U transformers datasets accelerate bitsandbytes

In [5]:
# import the libraries

from transformers import pipeline

# import the libraries

import torch
import torch.nn as nn
from transformers import DistilBertModel, DistilBertTokenizer, AdamW
from datasets import load_dataset
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence
from torch.optim import AdamW as aw

import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer

import matplotlib.pyplot as plt # this can be used to display loss/epoch graphs
from sklearn.model_selection import learning_curve

from sklearn.metrics import confusion_matrix # so i can display the confusion matrix

from torch.nn.utils.rnn import pad_sequence  # have to add padding library to ensure all sequences have the same length

import seaborn as sb

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # prepares GPU

# Load the dataset
dataset = load_dataset('emotion', trust_remote_code=True) # have to use the second parameter as there is required custom code for the dataset to be loaded properly

# Extract text sequences
texts = dataset['train']['text']


from transformers import pipeline

model_name = "distilgpt2"
generator = pipeline('text-generation', model=model_name)

prompt = "I'd like to write a poem about the beauty of nature."
result = generator(prompt, max_length=50, num_return_sequences=3)

for output in result:
    print(output['generated_text'])


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


I'd like to write a poem about the beauty of nature.
I'd like to write a poem about the beauty of nature. I'd like to include my thoughts on nature not having to choose to put down my own natural world as well as what my own life is full of.
I'd like to write
I'd like to write a poem about the beauty of nature. But there's actually no problem if you can't find out something about it, then the poetry is simply, 'You don't want a beautiful woman to have babies' or 'You


In [7]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [ ]:
from transformers import Trainer, TrainingArguments


from transformers import GPT2ForSequenceClassification

model = GPT2ForSequenceClassification.from_pretrained("gpt2", num_labels=2)

training_args = TrainingArguments(
    output_dir="./outputs",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
)

trainer.train()

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
